In [1]:
print("Hello")

Hello


In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
#this is config.py
import os
from dotenv import load_dotenv

load_dotenv()
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
# MODEL_NAME = os.getenv("MODEL_NAME", "gpt-3.5-turbo")

In [ ]:
%pip install langchain

In [4]:
#prompts.py
from langchain.prompts import PromptTemplate

GREETING_PROMPT = PromptTemplate.from_template(
    "Welcome to TalentScout! I am your Hiring Assistant. "
    "Before we proceed, please review our privacy policy and consent to data collection. "
    "Type 'I consent' to continue."
)

INFO_COLLECTION_PROMPT = PromptTemplate.from_template(
    "Let's get started. Please provide the following details:\n"
    "Full Name:\nEmail Address:\nPhone Number:\nYears of Experience:\n"
    "Desired Position(s):\nCurrent Location:"
)

TECH_STACK_PROMPT = PromptTemplate.from_template(
    "Please list the programming languages, frameworks, databases, and tools you are proficient in."
)

TECH_QUESTION_PROMPT = PromptTemplate.from_template(
    "Generate {num_questions} technical interview questions for a candidate skilled in {tech_stack}. "
    "Questions should assess both practical and conceptual knowledge."
)

THANK_YOU_PROMPT = PromptTemplate.from_template(
    "Thank you for your responses. We will review your application and contact you about next steps. Goodbye!"
)

FALLBACK_PROMPT = PromptTemplate.from_template(
    "I'm sorry, I didn't understand that. Could you please rephrase or provide more details?"
)

In [ ]:
%pip install langchain-together

In [111]:
res = llm.invoke("who are you")

In [112]:
res

AIMessage(content='I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 38, 'total_tokens': 61, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'cached_tokens': 0}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free', 'system_fingerprint': None, 'id': 'nxHkDQh-4Yz4kd-94c7df275c145a24', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--e5f58946-c133-4b23-bb2c-eb55e90726c6-0', usage_metadata={'input_tokens': 38, 'output_tokens': 23, 'total_tokens': 61, 'input_token_details': {}, 'output_token_details': {}})

In [113]:
%pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [114]:
#data_handler.py
import os
import json
from typing import Dict,Any
from cryptography.fernet import Fernet

FERNET_KEY = os.environ.get("FERNET_KEY")
if FERNET_KEY is None:
    FERNET_KEY = Fernet.generate_key().decode()
fernet = Fernet(FERNET_KEY.encode())

def encrypt_data(data: Dict[str,Any]) -> bytes:
    json_data = json.dumps(data).encode()
    return fernet.encrypt(json_data)

def decrypt_data(token: bytes)-> dict:
    return json.loads(fernet.decrypt(token).decode())

def save_candidate_data(candidate_data: Dict[str,Any], filename:str = "candidate_data.enc") -> None:
    encrypted = encrypt_data(candidate_data)
    with open(filename, "wb") as f:
        f.write(encrypted)

def load_candidate_data(filename:str = "candidate_data.enc") -> Dict[str,Any]:
    with open(filename,"rb") as f:
        encrypted = f.read()
    return decrypt_data(encrypted)

In [115]:
%pip install langgraph

Note: you may need to restart the kernel to use updated packages.


In [14]:
#state.py
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from typing import TypedDict, List,Annotated

class CandidateInfo(TypedDict):
    name: str
    email: str
    phone: str
    years_experience: str
    desired_position: str
    location: str

class ChatbotState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    candidate_info: CandidateInfo
    tech_stack: List[str]
    questions: List[str]
    current_question: int
    answers: List[str]


In [15]:
from typing import Union, Dict, Any
from langgraph.graph import StateGraph, END

In [16]:
# #graph.py
from typing import Union, Dict, Any
from langgraph.graph import StateGraph, END

# #Node Functions
# def greet_and_consent(state: ChatbotState):
#     last_msg = state["messages"][-1]
#     user_input = last_msg.content  # Get the actual text

#     if "consent" in user_input.lower():
#         return "collect_info", {}
    
#     return {"messages": GREETING_PROMPT}

# def collect_candidate_info(state: ChatbotState):
#     user_input = state["messages"][-1].content
#     info = [line.strip() for line in user_input.split('\n') if line.strip()]
#     if len(info) >= 6:
#         state["candidate_info"] = {
#             "name": info[0], "email": info[1], "phone": info[2],
#             "years_experience": info[3], "desired_position": info[4],
#             "location": info[5]
#         }
#     return {"messages": INFO_COLLECTION_PROMPT}


# def collect_tech_stack(state: ChatbotState):
#     user_input = state["messages"][-1].content
#     tech_stack = [t.strip() for t in user_input.split(',') if t.strip()]
#     if tech_stack:
#         state["tech_stack"] = tech_stack
#     return {"messages": TECH_QUESTION_PROMPT}




# def generate_tech_questions_node(state: ChatbotState):
#     questions = generate_tech_questions(state["tech_stack"])
#     state["questions"] = questions
#     state["current_question"] = 0
#     return {"messages": questions[0] if questions else "No questions generated."}

# def handle_qa(state: ChatbotState) -> Union[str, Dict[str, Any]]:
#     user_input = state["messages"][-1].content
#     answers = state.get("answers", []) + [user_input]
#     state["answers"] = answers
#     state["current_question"] += 1
#     if state["current_question"] < len(state["questions"]):
#         msg = state["questions"][state["current_question"]]
#         return {"messages": msg}
#     return "end"


# def fallback_response(state: ChatbotState):
#     return {"messages": FALLBACK_PROMPT}

# def end_conversation(state: ChatbotState):
#     return {"messages": THANK_YOU_PROMPT}

# #Build Workflow
# workflow = StateGraph(state_schema=ChatbotState)
# workflow.add_node("greet_and_consent",greet_and_consent)
# workflow.add_node("collect_info",collect_candidate_info)
# workflow.add_node("collect_tech_stack",collect_tech_stack)
# workflow.add_node("generate_questions",generate_tech_questions_node)
# workflow.add_node("handle_answers",handle_qa)
# workflow.add_node("fallback",fallback_response)
# workflow.add_node("end",end_conversation)


# workflow.add_edge("greet_and_consent","collect_info")
# workflow.add_edge("collect_info","collect_tech_stack")
# workflow.add_edge("collect_tech_stack","generate_questions")
# workflow.add_edge("generate_questions","handle_answers")
# workflow.add_edge("handle_answers","end")
# workflow.add_edge("fallback","end")
# workflow.set_entry_point("greet_and_consent")

# compiled_workflow = workflow.compile()


In [17]:
#tech_questions.py
from langchain_together import ChatTogether

llm = ChatTogether(model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free")

# def generate_tech_questions(tech_stack,num_questions = 3):
#     prompt = TECH_QUESTION_PROMPT.format(num_questions=num_questions,tech_stack = ", ".join(tech_stack))
#     questions = llm(prompt)
#     if isinstance(questions, str):
#         return [q.strip() for q in questions.split('\n') if q.strip()]
#     return questions

# def generate_tech_questions(tech_stack, num_questions=3):
#     prompt = TECH_QUESTION_PROMPT.format(num_questions=num_questions, tech_stack=", ".join(tech_stack))
#     questions = llm(prompt)
#     print(f"These are questions {questions}")
#     # Ensure questions is a list of strings
#     if isinstance(questions, str):
#         # Split by newline and remove empty lines
#         return [q.strip() for q in questions.split('\n') if q.strip()]
#     return questions

In [43]:
from langchain_core.messages import HumanMessage

TECH_QUESTION_PROMPT = (
    "Generate {num_questions} technical interview questions for a candidate skilled in {tech_stack}. "
    "Return ONLY a valid Python list of strings, with each question as a separate string in the list. "
    "Do not include any explanations, formatting, or code blocks—just the Python list."
)


def generate_tech_questions(tech_stack, num_questions=3):
    prompt = TECH_QUESTION_PROMPT.format(num_questions=num_questions, tech_stack=", ".join(tech_stack))
    # Wrap prompt in a HumanMessage and pass as a list
    response = llm.invoke([HumanMessage(content=prompt)])
    # The model's response is a message object; get its .content
    questions = response.content
    return questions

# Example usage:
question_list = generate_tech_questions(['Python', 'java'])
print(question_list)

["How would you implement a queue using a linked list in Python", "Write a Java program to find the maximum and minimum values in a given array", "Explain how you would optimize the performance of a Python function that is currently using excessive memory"]


In [34]:
import ast

In [36]:
try:
    question_list = ast.literal_eval(question_list)
except Exception as e:
    print(f"Error parsing LLM output: {e}")
    question_list = []

print(question_list)  # This will now print the first question, not '['

Error parsing LLM output: malformed node or string: []
[]


In [44]:
import ast

# This could be a string (from LLM) or a list (already parsed)
llm_output = question_list  # This might be a string or a list

# Robust parsing
if isinstance(llm_output, str):
    try:
        question_list = ast.literal_eval(llm_output)
    except Exception as e:
        print(f"Error parsing LLM output: {e}")
        question_list = []
elif isinstance(llm_output, list):
    question_list = llm_output
else:
    question_list = []

# Now you can safely use question_list[0]
if question_list:
    print(question_list[0])
else:
    print("No questions found.")


How would you implement a queue using a linked list in Python


In [37]:
import ast

llm_output = '["How would you implement a thread-safe queue in Python to handle concurrent access from multiple threads", "Write a Java program to find the middle element of a singly linked list", "Explain how you can use Python\'s decorators to implement aspect-oriented programming and provide an example of logging function calls"]'

try:
    question_list = ast.literal_eval(llm_output)
except Exception as e:
    print(f"Error parsing LLM output: {e}")
    question_list = []

print(question_list[0])  # This will now print the first question, not '['

How would you implement a thread-safe queue in Python to handle concurrent access from multiple threads


In [32]:
question_list[0]

'['

In [12]:
question = generate_tech_questions(['Python', 'java'])

In [19]:
from langchain_core.messages import AIMessage

In [51]:
from pprint import pprint

In [55]:
GREETING_PROMPT = "Welcome to TalentScout! I am your Hiring Assistant. Before we proceed, please review our privacy policy and consent to data collection. Type 'I consent' to continue."
INFO_COLLECTION_PROMPT = "Let's get started. Please provide the following details:\nFull Name:\nEmail Address:\nPhone Number:\nYears of Experience:\nDesired Position(s):\nCurrent Location:"
TECH_STACK_PROMPT = "Please list the programming languages, frameworks, databases, and tools you are proficient in."
FALLBACK_PROMPT = "I'm sorry, I didn't understand that. Could you please rephrase or provide more details?"
THANK_YOU_PROMPT = "Thank you for your responses. We will review your application and contact you about next steps. Goodbye!"

class CandidateInfo(TypedDict):
    name: str
    email: str
    phone: str
    years_experience: str
    desired_position: str
    location: str

class ChatbotState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    candidate_info: CandidateInfo
    tech_stack: List[str]
    questions: List[str]
    current_question: int
    answers: List[str]


def greet_and_consent(state):
    last_msg = state["messages"][-1]
    print(f"{state['messages']}")
    user_input = input("You: ")
    # user_input = last_msg.content
    if "consent" in user_input.lower():
        print("consent")
        return {**state, "next": "collect_info"}
    new_msg = AIMessage(content=GREETING_PROMPT)
    print("after consent")
    return {**state, "messages": state["messages"] + [new_msg]}

def collect_candidate_info(state):
    print("collect _candidate stated")
    print("Provide user you information like name ,email, phone, years of experince desired_position, location each in new line")
    name = input('Enter your name: ')
    email = input('Enter your email: ')
    phone = input('Enter your phone number: ')
    years_experience = input('Enter your years of experience: ')
    desired_position = input('Enter your desired position: ')
    location = input('Enter your current location: ')
    # Build the candidate_info dict
    candidate_info = {
        "name": name,
        "email": email,
        "phone": phone,
        "years_experience": years_experience,
        "desired_position": desired_position,
        "location": location
    }

    # Update your state
    state["candidate_info"] = candidate_info
    # print("content accessed")
    # info = [line.strip() for line in user_input.split('\n') if line.strip()]
    # if len(info) >= 6:
    #     candidate_info = {
    #         "name": info[0], "email": info[1], "phone": info[2],
    #         "years_experience": info[3], "desired_position": info[4],
    #         "location": info[5]
    #     }
    return {**state, "candidate_info": candidate_info}
    # new_msg = AIMessage(content=INFO_COLLECTION_PROMPT)
    # return {**state, "messages": state["messages"] + [new_msg], "next": "collect_info"}

# def collect_tech_stack(state):
#     print("Collect techs tack started")
#     user_input = input("You : ")
    
#     # last_msg = state["messages"][-1]
#     # user_input = last_msg.content
#     tech_stack = [t.strip() for t in user_input.split(',') if t.strip()]
#     if tech_stack:
#         return {**state, "tech_stack": tech_stack, "next": "generate_questions"}
#     new_msg = AIMessage(content=TECH_STACK_PROMPT)
#     return {**state, "messages": state["messages"] + [new_msg]}
def collect_tech_stack(state):
    print("Collect tech stack started")
    user_input = input("You: ")
    if user_input.strip().lower() == "exit":
        return {**state, "exit": True, "next": "end"}
    tech_stack = [t.strip() for t in user_input.split(',') if t.strip()]
    if not tech_stack:
        print("Error: No tech stack provided.")
        return {**state, "error": True, "next": "collect_tech_stack"}
    return {**state, "tech_stack": tech_stack, "next": "generate_questions"}


def generate_tech_questions_node(state):
    print("generete tech questions started")
    print(f"{state.get('tech_stack')}")
    questions = generate_tech_questions(state.get("tech_stack", []))
    # print(questions)
    first_question = questions[0] if questions else "No questions generated."
    new_msg = AIMessage(content=first_question)
    return {
        **state,
        "questions": questions,
        "current_question": 0,
        "messages": state["messages"] + [new_msg],
        "next": "handle_answers"
    }

def ask_questions_and_collect_answers(state):
    """
    Loops through each technical question, displays it, gets user input,
    and stores all answers in the state["answers"] list.
    """
    questions = state.get("questions", [])
    print(f"these are your questions {questions}")
    if not questions:
        print("No questions to ask.")
        return state
    
    if isinstance(questions, str):
        try:
            question_list = ast.literal_eval(questions)
        except Exception as e:
            print(f"Error parsing LLM output: {e}")
            question_list = []
    elif isinstance(questions, list):
        question_list = questions
    else:
        question_list = []

    # Now you can safely use question_list[0]
    if question_list:
        answers = []
        for idx, question in enumerate(question_list):
            print(f"Question {idx}: {question}")
            user_answer = input("Your answer: ")
            answers.append(user_answer)
            # Optionally, store the Q&A as a message for conversation history
            state["messages"].append(AIMessage(content=question))
            state["messages"].append(HumanMessage(content=user_answer))
    else:
        print("No questions found.")

    # answers = []
    # for idx, question in enumerate(questions):
    #     print(f"Question {idx}: {question}")
    #     user_answer = input("Your answer: ")
    #     answers.append(user_answer)
    #     # Optionally, store the Q&A as a message for conversation history
    #     state["messages"].append(AIMessage(content=question))
    #     state["messages"].append(HumanMessage(content=user_answer))

    # state["answers"] = answers
    state["current_question"] = len(questions)  # All questions answered
    return state


def fallback_response(state):
    new_msg = AIMessage(content=FALLBACK_PROMPT)
    print(FALLBACK_PROMPT)
    return {**state, "messages": state["messages"] + [new_msg], "next": "fallback"}

def end_conversation(state):
    new_msg = AIMessage(content=THANK_YOU_PROMPT)
    print("=== Raw State ===")
    pprint(state)
    return {**state, "messages": state["messages"] + [new_msg], "next": "end"}


#Build Workflow
workflow = StateGraph(state_schema=ChatbotState)
workflow.add_node("greet_and_consent",greet_and_consent)
workflow.add_node("collect_info",collect_candidate_info)
workflow.add_node("collect_tech_stack",collect_tech_stack)
workflow.add_node("generate_questions",generate_tech_questions_node)
workflow.add_node("handle_answers",ask_questions_and_collect_answers)
workflow.add_node("fallback",fallback_response)
workflow.add_node("end",end_conversation)


workflow.add_edge("greet_and_consent","collect_info")
workflow.add_edge("collect_info","collect_tech_stack")
workflow.add_edge("collect_tech_stack","generate_questions")
workflow.add_edge("generate_questions","handle_answers")
workflow.add_edge("handle_answers","fallback")
workflow.add_edge("fallback","end")
workflow.set_entry_point("greet_and_consent")

compiled_workflow = workflow.compile()


In [56]:
input_message = {"role": "user", "content": "Hi"}
initial_state = {
    "messages": [input_message],
    "answers": [],
    "current_question": 0,
    "questions": [],
    "tech_stack": [],
    "candidate_info": {
        "name": "", "email": "", "phone": "",
        "years_experience": "", "desired_position": "", "location": ""
    }
}


In [57]:
result = compiled_workflow.invoke(initial_state)

[HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='30212a3b-7487-4592-8e31-db9a50e7fe41')]


consent
collect _candidate stated
Provide user you information like name ,email, phone, years of experince desired_position, location each in new line
Collect tech stack started
generete tech questions started
[]
these are your questions ["Explain how you would implement a singleton class in Python, ensuring thread safety and lazy initialization", "Write a Python function to find the first duplicate in an array of integers, optimizing for time complexity", "Describe a strategy for optimizing the performance of a Python application that relies heavily on file I/O operations, including caching and buffering techniques"]
Question 0: Explain how you would implement a singleton class in Python, ensuring thread safety and lazy initialization
Question 1: Write a Python function to find the first duplicate in an array of integers, optimizing for time complexity
Question 2: Describe a strategy for optimizing the performance of a Python application that relies heavily on file I/O operations, inc

In [26]:
from langchain_core.messages import HumanMessage

state = {
    "messages": [HumanMessage(content="Hi I am here for interview")],
    "answers": [],
    # You can omit tech_stack, questions, etc., and they'll be added during execution
}

res = compiled_workflow.invoke(state)

[HumanMessage(content='Hi I am here for interview', additional_kwargs={}, response_metadata={}, id='d433f064-bc46-4c7f-88d8-fadb4bdc7cd4')]
consent
collect _candidate stated
Provide user you information like name ,email, phone, years of experince desired_position, location each in new line
Collect techs tack started
generete tech questions started
['Python', 'java']
["How would you implement a thread-safe queue in Python", "Write a Java program to find the middle element of a singly linked list", "Explain how you would optimize the performance of a Python function that is making multiple API calls to an external service"]
these are your questions ["How would you implement a thread-safe queue in Python", "Write a Java program to find the middle element of a singly linked list", "Explain how you would optimize the performance of a Python function that is making multiple API calls to an external service"]


In [ ]:
graph

'---\nconfig:\n  flowchart:\n    curve: linear\n---\ngraph TD;\n\t__start__(<p>__start__</p>)\n\tgreet_and_consent(greet_and_consent)\n\tcollect_info(collect_info)\n\tcollect_tech_stack(collect_tech_stack)\n\tgenerate_questions(generate_questions)\n\thandle_answers(handle_answers)\n\tfallback(fallback)\n\tend(end)\n\t__end__(<p>__end__</p>)\n\t__start__ --> greet_and_consent;\n\tcollect_info --> collect_tech_stack;\n\tcollect_tech_stack --> generate_questions;\n\tgenerate_questions --> handle_answers;\n\tgreet_and_consent --> collect_info;\n\thandle_answers --> end;\n\tend --> __end__;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n'

In [ ]:
res = compiled_workflow.invoke({"messages":"Hi i am here for interview"})

InvalidUpdateError: Expected dict, got ('greet_and_consent', {'message': PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Welcome to TalentScout! I am your Hiring Assistant. Before we proceed, please review our privacy policy and consent to data collection. Type 'I consent' to continue.")})
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

In [ ]:
# Test the graph workflow with a demo interview

test_state = {}
current_node = "greet_and_consent"

# Simulated user inputs for each step
user_inputs = [
    "I consent",  # Consent
    "John Doe\njohn@example.com\n1234567890\n5\nSoftware Engineer\nNew York",  # Candidate info
    "Python, Django, PostgreSQL",  # Tech stack
    "Answer to Q1",  # Tech Q1
    "Answer to Q2",  # Tech Q2
    "Answer to Q3",  # Tech Q3
]

print("=== Demo Interview ===")
for user_input in user_inputs:
    node_func = compiled_workflow.get_node(current_node)
    next_node, response = node_func(test_state, user_input)
    print(f"\nYou: {user_input}")
    if isinstance(response, dict):
        message = response.get("message", "")
        print(f"Bot: {message}")
        test_state.update(response)
    else:
        print(f"Bot: {response}")
    current_node = next_node
    if current_node == "end":
        # Final message
        node_func = compiled_workflow.get_node(current_node)
        _, response = node_func(test_state, "")
        if isinstance(response, dict):
            print(f"Bot: {response.get('message', '')}")
        else:
            print(f"Bot: {response}")
        break

=== Demo Interview ===


AttributeError: 'CompiledStateGraph' object has no attribute 'get_node'

In [ ]:
# Test the graph workflow with a demo interview

test_state = {}
current_node = "greet_and_consent"

# Simulated user inputs for each step
user_inputs = [
    "I consent",  # Consent
    "John Doe\njohn@example.com\n1234567890\n5\nSoftware Engineer\nNew York",  # Candidate info
    "Python, Django, PostgreSQL",  # Tech stack
    "Answer to Q1",  # Tech Q1
    "Answer to Q2",  # Tech Q2
    "Answer to Q3",  # Tech Q3
]

print("=== 🧪 Demo Interview ===")
for user_input in user_inputs:
    # Use invoke instead of get_node
    next_node, response = compiled_workflow.invoke(current_node, test_state, user_input=user_input)

    print(f"\n🧑 You: {user_input}")
    
    # Display bot's message
    if isinstance(response, dict):
        message = response.get("message", "")
        print(f"🤖 Bot: {message}")
        test_state.update(response)
    else:
        print(f"🤖 Bot: {response}")
    
    current_node = next_node
    
    if current_node == "end":
        # Final message
        _, final_response = compiled_workflow.invoke(current_node, test_state)
        if isinstance(final_response, dict):
            print(f"🤖 Bot: {final_response.get('message', '')}")
        else:
            print(f"🤖 Bot: {final_response}")
        break


=== 🧪 Demo Interview ===


TypeError: Pregel.stream() got an unexpected keyword argument 'user_input'

In [ ]:
%pip install streamlit

In [ ]:
import streamlit as st
# from .chatbot_graph import workflow

st.title("TalentScout Hiring Assistant")

if "state" not in st.session_state:
    st.session_state.state = {}
    st.session_state.node = "greet_and_consent"
    st.session_state.history = []

user_input = st.text_input("You:", key="user_input")

if st.button("Send") or user_input:
    # Use compiled workflow to get the next node and response
    node_func = compiled_workflow.get_node(st.session_state.node)
    next_node, response = node_func(st.session_state.state, user_input)
    st.session_state.node = next_node
    if isinstance(response, dict):
        st.session_state.state.update(response)
        st.session_state.history.append(("Bot", response.get("message", "")))
    else:
        st.session_state.history.append(("Bot", str(response)))
    st.session_state.history.append(("You", user_input))
    st.session_state.user_input = ""  # Clear input

for speaker, message in st.session_state.history:
    st.markdown(f"**{speaker}:** {message}")

2025-06-08 09:56:29.760 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-08 09:56:29.932 
  command:

    streamlit run c:\Users\Dikshit\miniconda3\envs\hiring-agent\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-06-08 09:56:29.933 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-08 09:56:29.933 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-08 09:56:29.933 Session state does not function when running a script without `streamlit run`
2025-06-08 09:56:29.934 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-08 09:56:29.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-08 09:56:29.935 Thread 'MainThread': missi